In [2]:
# Packages needed for plots and analysis. 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import itertools as it
import seaborn as sns
from pyrolite.plot import pyroplot
import plotly.express as px
from scipy import stats

In [3]:
cols = ['Material', 'Sample', 'Site','Fe2O3 (%)' ]
df=pd.read_csv("./data_sin.csv", sep = ",", usecols = cols)
df = df.loc[(~df["Material"].isin(["Sandstone", "Bloom","Smithing Slag", "Iron-SI","Iron-SI-P"])) & \
           (~df["Site"].isin(["Imbasch", "Stumpfwald"]))]
p = 2*55.8/(2*55.8+3*16)
df["Fe %"] = p * df["Fe2O3 (%)"]

print(df)

      Material     Sample              Site  Fe2O3 (%)       Fe %
1          Ore  MA-200975         Eisenberg      70.78  49.492782
3          Ore  MA-200977         Eisenberg      72.00  50.345865
4    Oven Slag  MA-200978         Eisenberg      63.27  44.241429
5    Oven Slag  MA-200979         Eisenberg      63.65  44.507143
6    Oven Slag  MA-200980         Eisenberg      54.46  38.081053
7    Oven Slag  MA-200981         Eisenberg      76.69  53.625338
8    Oven Slag  MA-200983         Eisenberg      48.14  33.661805
13  Plate Slag  MA-200989         Eisenberg      64.31  44.968647
14  Plate Slag  MA-200990         Eisenberg      77.79  54.394511
15  Plate Slag  MA-200991         Eisenberg      86.50  60.484962
16  Plate Slag  MA-200992         Eisenberg      47.26  33.046466
17  Plate Slag  MA-200994         Eisenberg      52.69  36.843383
18  Plate Slag  MA-200995         Eisenberg      47.56  33.256241
19  Plate Slag  MA-200996         Eisenberg      57.80  40.416541
20  Plate 

In [4]:
mask = (df["Material"] == "Ore")
df1 = df.loc[~mask]
result_slag = df1.groupby(['Material'], as_index = False).agg({"Fe %": ['mean', "min", "max"]})
result_slag.columns = pd.MultiIndex.from_tuples((("Slag", ""), ("Fe % in slag", "mean"), ("Fe % in slag", "min"), ("Fe % in slag", "max")))
result_slag

Slag Fe % in slag                      
                      mean        min        max
0   Flow Slag    40.715947  33.368120  51.184962
1   Oven Slag    42.823353  33.661805  53.625338
2  Plate Slag    41.669594  32.256316  60.484962

In [5]:
mask = (df["Material"] != "Ore")
df2 = df.loc[~mask].drop(columns = "Fe2O3 (%)")
result_ore = df2.groupby(['Site'], as_index = False).agg({"Fe %": ['mean', "min", "max"]})
result_ore.columns = pd.MultiIndex.from_tuples((("Ore Site", ""), ("Fe % in ore", "mean"), ("Fe % in ore", "min"), ("Fe % in ore", "max")))
result_ore

Ore Site Fe % in ore                      
                           mean        min        max
0         Eisenberg   50.066165  49.492782  50.359850
1  Hettenleidelheim   50.828346  43.765940  59.526992
2        Tiefenthal   45.318271  44.395263  46.241278

In [6]:
pf = 98
result_slag["k"] = 1
result_ore["k"] = 1
result = pd.merge(result_slag, result_ore, on = "k").drop(columns = "k")
result[("Ef", "mean")] = 0
result[("Ef", "max")] = 0
result[("Ef", "mean")] = (pf / result[("Fe % in ore", "mean")])*((result[("Fe % in ore", "mean")] - result[("Fe % in slag", "mean")]) / (pf - result[("Fe % in slag", "mean")]))
result[("Ef", "max")] = (pf / result[("Fe % in ore", "max")])*((result[("Fe % in ore", "max")] - result[("Fe % in slag", "min")]) / (pf - result[("Fe % in slag", "min")]))
result

/home/marisa/.local/lib/python3.8/site-packages/pandas/core/generic.py:4147: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Slag Fe % in slag                                Ore Site  \
                      mean        min        max                     
0   Flow Slag    40.715947  33.368120  51.184962         Eisenberg   
1   Flow Slag    40.715947  33.368120  51.184962  Hettenleidelheim   
2   Flow Slag    40.715947  33.368120  51.184962        Tiefenthal   
3   Oven Slag    42.823353  33.661805  53.625338         Eisenberg   
4   Oven Slag    42.823353  33.661805  53.625338  Hettenleidelheim   
5   Oven Slag    42.823353  33.661805  53.625338        Tiefenthal   
6  Plate Slag    41.669594  32.256316  60.484962         Eisenberg   
7  Plate Slag    41.669594  32.256316  60.484962  Hettenleidelheim   
8  Plate Slag    41.669594  32.256316  60.484962        Tiefenthal   

  Fe % in ore                              Ef            
         mean        min        max      mean       max  
0   50.066165  49.492782  50.359850  0.319499  0.511602  
1   50.828346  43.765940  59.526992  0.340362  0.666322  
2   45.318271  44.395263  46.241278  0.173739  0.422119  
3   50.066165  49.492782  50.359850  0.256941  0.505055  
4   50.828346  43.765940  59.526992  0.279721  0.661849  
5   45.318271  44.395263  46.241278  0.097781  0.414371  
6   50.066165  49.492782  50.359850  0.291770  0.535860  
7   50.828346  43.765940  59.526992  0.313483  0.682895  
8   45.318271  44.395263  46.241278  0.140070  0.450820

In [7]:
#result.to_csv("efficiency.csv", sep = ";")